# Titanic - Machine Learning

## Analíse exploratória inicial com **ydata-profiling**.

1. Gera um relatório inicial a partir do DataFrame do Pandas;
2. Importante para analisar quais colunas possuem valores vazios, por exemplo.

In [1]:
import pandas as pd

# Visualizando a base de treino
treino = pd.read_csv('DataBase/train.csv') # Carregando a base de dados de treino como um dataframe do Pandas

treino.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [2]:
# Relatório inicial gerado por ydata-profiling
from ydata_profiling import ProfileReport

profile = ProfileReport(treino, title="Relatório da Base de Treino do Titanic", explorative=True)
profile.to_file("Relatório da Base de Treino do Titanic.html")

c:\Users\Marino\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 50.00it/s]


* Analisando a base de dados de treino

In [3]:
# Verificando as informações da base de treino
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
# Verificando o tipo de dados que há no database de treino
treino.dtypes.value_counts()

int64      5
object     5
float64    2
Name: count, dtype: int64

In [5]:
# Contagem de valores nulos
treino.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

* Analisando a base de dados para realizar testes de predição

In [6]:
# Visualizando a base de teste para exploração
teste = pd.read_csv('DataBase/test.csv')
teste.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [7]:
teste.info()
teste.isnull().sum().sort_values(ascending=False).head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
dtype: int64

## Tratamentos iniciais dos dados

* É necessário entender e analisar os dados nas duas bases de dados disponíveis.

In [8]:
treino.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

In [9]:
treino.shape

(891, 12)

In [10]:
teste.shape

(418, 11)

* Muitas linhas da base de treino estão vazias e não há uma padrão nos valores em uma mesma coluna

In [11]:
treino.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [12]:
# Verificando a cardinalidade dos dados
treino.nunique().sort_values(ascending=False)

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
Age             88
SibSp            7
Parch            7
Pclass           3
Embarked         3
Survived         2
Sex              2
dtype: int64

* Colunas com alta cardialidade, isto é, com muitos valores diferentes (Name, Ticket, Cabin) atrapalham o nosso modelo de predição por Machine Learning. A ideia aqui é tornar os dados limpos para deixar o modelo menos generalizável. Uma possibilidade é eliminar estas colunas.

* Name, Ticket e Cabin não são tão úteis para a nossa predição do modelo, então podem ser excluídas. Já Age é um atributo interessante para se analisar e deve ser visto com cuidado, pois possui valores nulos.

In [13]:
treino = treino.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# Sem o inplace=True, a alteração não é salva no dataframe original.

In [14]:
teste = teste.drop(['Name', 'Ticket', 'Cabin'], axis=1)

* Analisando a coluna Age para tratar valores vazios: uma ideia é substituir tais espaços pela média das idades.

In [15]:
# Verificando valores ausentes em Ages -> Não podemos ter valores nulos, e excluir estes passageiros seria ruim para o modelo. 
# Uma alternativa é preencher esses valores com a mediana ou média das idades.

treino.Age.mean()

29.69911764705882

In [15]:
treino.loc[treino.Age.isnull()] # Todas as linhas que possuem valores nulos em Age

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,6,0,3,male,NaN,0,0,8.4583,Q
17,18,1,2,male,NaN,0,0,13.0000,S
19,20,1,3,female,NaN,0,0,7.2250,C
26,27,0,3,male,NaN,0,0,7.2250,C
28,29,1,3,female,NaN,0,0,7.8792,Q
...,...,...,...,...,...,...,...,...,...
859,860,0,3,male,NaN,0,0,7.2292,C
863,864,0,3,female,NaN,8,2,69.5500,S
868,869,0,3,male,NaN,0,0,9.5000,S
878,879,0,3,male,NaN,0,0,7.8958,S


In [16]:
treino.loc[treino.Age.isnull(), 'Age'] = treino.Age.mean() # Preenchendo os valores nulos com a média

* O mesmo pensamento se aplica à base de teste, devemos preencher com a média das idades as linhas na coluna Age que possuirem valores nulos.

In [ ]:
teste.loc[teste.Age.isnull()] # Todas as linhas que possuem valores nulos em Age

teste.loc[teste.Age.isnull(), 'Age'] = teste.Age.mean() # Preenchendo os valores nulos com a média da base de teste

* Há também valores vazios na coluna "Embakerd", mas como também não queremos excluir a linha e essa coluna pode ser relavante pro nosso modelo, vamos substituir pela Moda

In [18]:
treino.Embarked.value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [19]:
treino.Embarked.mode()[0]

'S'

In [20]:
treino.loc[treino.Embarked.isnull(), 'Embarked'] = treino.Embarked.mode()[0]

* Há valores de tarifa (Fare) nulos, mas podemos preencher pela estatística mais relevante, neste caso, a média é útil para o nosso problema.

In [21]:
teste.loc[teste.Fare.isnull(), 'Fare'] = teste.Fare.mean()

In [22]:
treino.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
dtype: int64

In [23]:
teste.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
dtype: int64

* Até aqui, analisamos apenas colunas que possuem valores do tipo _string_ e 'object'. 

* Podemos começar a analisar colunas que possuem valores numéricos.

In [24]:
treino.columns[treino.dtypes != 'object'] # Selecionando colunas que não são do tipo string

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [ ]:
treino_nr = treino.loc[:, treino.columns[treino.dtypes != 'object']] # Base de treino numérica

In [ ]:
teste.columns[teste.dtypes != 'object'] # Selecionando colunas que não são do tipo string
teste_nr = teste.loc[:, teste.columns[teste.dtypes != 'object']] # Base de teste numérica

## Modelos de Classificação por Machine Learning

### Modelos disponíveis pelo scikit-learn considerados aqui:
1. Árvore de classificação (Tree)
2. Classificação dos vizinhos mais próximos (KNN)
3. Regressão Logistíca

* Antes de utilizar e testar os algoritmos, precisamos separar a base de treino e validação dos modelos utilizando o **train_test_split** do **sklearn**
    - Este módulo consiste em dividir um conjunto de dados em dois subconjuntos: um para treinamento e outro para teste. O conjunto de treinamento é usado para treinar um modelode Machine Learning, enquanto o conjunto de teste é usado para avaliar seu desempenho.

In [27]:
# Importanto Scikit-Learn
import sklearn
from sklearn.model_selection import train_test_split

In [28]:
# Separando a base de treino em features e target
x = treino_nr.drop(['PassengerId', 'Survived'], axis=1) # Base de treino numéricas retirando as colunas que quero prever
y = treino.Survived

In [29]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.33, random_state=42)

* Aqui, $x$ e $y$ são o conjunto de dados a ser dividido, **test_size** é a proporção do conjunto de dados a ser alocado ao **conjunto de teste**, isto é, é como os dados serão divididos a partir da proporção estabelecida. Os dados restante são usados para treinamento, após a divisão.

* É importante notar que, a geração dos conjuntos de teste e treinamento são aleatórias e podem mudar a cada vez que o código é executado. Por isso, é necessário o atributo **random_state**
    - **random_state** é um parâmetro que controla o gerador de números aleatórios usado para embaralhar os dados antes de dividi-los. Em outras palavras, ele garante que a mesma randomização seja usada sempre que o código for executado.

### Árvore de Decisão

In [30]:
from sklearn import tree

clf_ac = tree.DecisionTreeClassifier(random_state=42)
clf_ac = clf_ac.fit(x_train, y_train)
y_pred_ac = clf_ac.predict(x_val)

### KNeighborsClassifier (KNN)

In [31]:
from sklearn.neighbors import KNeighborsClassifier

clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn = clf_knn.fit(x_train, y_train)
y_pred_knn = clf_knn.predict(x_val)

### Regressão Logística

In [32]:
from sklearn.linear_model import LogisticRegression

clf_rl = LogisticRegression(random_state=42)
clf_rl = clf_rl.fit(x_train, y_train)
y_pred_rl = clf_rl.predict(x_val)

### Qual é o melhor modelo para a nossa análise?

* Devemos avaliar os modelos a partir de métricas, disponíveis no sklearn. Aqui, vamos considerar o modelo que tiver a maior Acurácia (taxa de acerto). Uma outra possibilidade seria considerar a distribuição de erros no modelo a partir da Matriz de Confusão.

In [33]:
from sklearn.metrics import accuracy_score

# Para a Árvore de Classificação
accuracy_score(y_val, y_pred_ac)

0.6169491525423729

In [34]:
# Para o método KNN
accuracy_score(y_val, y_pred_knn)

0.6542372881355932

In [35]:
# Para o método de Regressão Logística
accuracy_score(y_val, y_pred_rl)

0.7254237288135593

* Aqui, o modelo de Regressão Logística possui a maior métrica de Acurácia, classificando-o para ser o nosso modelo de predição.

* Para estudo, podemos também utilizar a métrica de Matriz de Confusão

In [33]:
from sklearn.metrics import confusion_matrix

# Para a Árvore de Classificação
confusion_matrix(y_val, y_pred_ac)

array([[125,  50],
       [ 63,  57]], dtype=int64)

In [34]:
# Para o KNN
confusion_matrix(y_val, y_pred_knn)

array([[133,  42],
       [ 60,  60]], dtype=int64)

In [35]:
# Para a Regressão Logística
confusion_matrix(y_val, y_pred_rl)

array([[156,  19],
       [ 62,  58]], dtype=int64)

### Fazendo previsão para os dados de teste com o modelo de predição com maior precisão (Acurácia)

In [36]:
x_train.head(3)

,Pclass,Age,SibSp,Parch,Fare
6,1,54.000000,0,0,51.8625
718,3,29.699118,0,0,15.5000
685,2,25.000000,1,2,41.5792


In [37]:
teste_nr.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,892,3,34.5,0,0,7.8292
1,893,3,47.0,1,0,7.0000
2,894,2,62.0,0,0,9.6875


* As duas bases de treino são diferentes devido a coluna PassergerId

In [38]:
x_teste = teste_nr.drop("PassengerId", axis=1)

In [39]:
teste.shape

(418, 8)

In [40]:
# Utilizando a Regressão Logística para fazer a previsão
y_pred = clf_rl.predict(x_teste)

In [41]:
# Criando uma nova coluna com a previsão da base de teste
teste_nr['Survived'] = y_pred

In [42]:
base_final = teste_nr[['PassengerId', 'Survived']]
base_final.to_csv('titanic_previsao.csv', index=False)